In [1]:
import numpy as np
import pandas as pd

In [2]:
timesteps = 14
startDay = 350

In [3]:
dt = pd.read_csv("m5-forecasting-accuracy/sales_train_evaluation.csv")
dt.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1


In [4]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int64(1941), object(6)
memory usage: 452.9+ MB


In [5]:
#To reduce memory usage
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [6]:
#Reduce memory usage and compare with the previous one to be sure
dt = downcast_dtypes(dt)

In [7]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int16(1941), object(6)
memory usage: 114.3+ MB


In [8]:
#Take the transpose so that we have one day for each row, and 30490 items' sales as columns
dt = dt.T    
dt.head(8)

,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
id,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_010_CA_1_evaluation,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
item_id,HOBBIES_1_001,HOBBIES_1_002,HOBBIES_1_003,HOBBIES_1_004,HOBBIES_1_005,HOBBIES_1_006,HOBBIES_1_007,HOBBIES_1_008,HOBBIES_1_009,HOBBIES_1_010,...,FOODS_3_818,FOODS_3_819,FOODS_3_820,FOODS_3_821,FOODS_3_822,FOODS_3_823,FOODS_3_824,FOODS_3_825,FOODS_3_826,FOODS_3_827
dept_id,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,...,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3
cat_id,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,...,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS
store_id,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,...,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3
state_id,CA,CA,CA,CA,CA,CA,CA,CA,CA,CA,...,WI,WI,WI,WI,WI,WI,WI,WI,WI,WI
d_1,0,0,0,0,0,0,0,12,2,0,...,0,14,1,0,4,0,0,0,0,0
d_2,0,0,0,0,0,0,0,15,0,0,...,0,11,1,0,4,0,0,6,0,0


In [9]:
#Remove id, item_id, dept_id, cat_id, store_id, state_id columns
dt = dt[6 + startDay:]
dt.head(5)

,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_351,0,0,0,2,0,0,0,24,3,2,...,0,9,1,0,11,0,0,1,0,0
d_352,0,0,0,0,0,0,0,9,0,2,...,0,5,4,0,8,0,1,2,0,0
d_353,0,0,0,4,2,0,0,2,1,1,...,0,15,2,0,3,0,1,2,0,0
d_354,0,1,0,2,0,0,0,7,1,0,...,0,5,1,0,3,0,0,0,0,0
d_355,0,0,0,1,2,0,0,0,0,0,...,0,7,1,0,1,0,1,1,0,0


In [10]:
calendar = pd.read_csv("m5-forecasting-accuracy/calendar.csv")
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [11]:
#Create dataframe with zeros for 1969 days in the calendar
daysBeforeEvent = pd.DataFrame(np.zeros((1969,1)))

In [12]:
# "1" is assigned to the days before the event_name_1. Since "event_name_2" is rare, it was not added.
for x,y in calendar.iterrows():
    if((pd.isnull(calendar["event_name_1"][x])) == False):
           daysBeforeEvent[0][x-1] = 1 
            #if first day was an event this row will cause an exception because "x-1".
            #Since it is not i did not consider for now.

In [13]:
#"calendar" won't be used anymore. 
del calendar

In [14]:
#"daysBeforeEventTest" will be used as input for predicting (We will forecast the days 1913-1941)
daysBeforeEventTest = daysBeforeEvent[1941:1969]
#"daysBeforeEvent" will be used for training as a feature.
daysBeforeEvent = daysBeforeEvent[startDay:1941]

In [15]:
#Before concatanation with our main data "dt", indexes are made same and column name is changed to "oneDayBeforeEvent"
daysBeforeEvent.columns = ["oneDayBeforeEvent"]
daysBeforeEvent.index = dt.index

In [16]:
dt = pd.concat([dt, daysBeforeEvent], axis = 1)

dt.columns

Index([                  0,                   1,                   2,
                         3,                   4,                   5,
                         6,                   7,                   8,
                         9,
       ...
                     30481,               30482,               30483,
                     30484,               30485,               30486,
                     30487,               30488,               30489,
       'oneDayBeforeEvent'],
      dtype='object', length=30491)

In [17]:
dt

,0,1,2,3,4,5,6,7,8,9,...,30481,30482,30483,30484,30485,30486,30487,30488,30489,oneDayBeforeEvent
d_351,0,0,0,2,0,0,0,24,3,2,...,9,1,0,11,0,0,1,0,0,0.0
d_352,0,0,0,0,0,0,0,9,0,2,...,5,4,0,8,0,1,2,0,0,1.0
d_353,0,0,0,4,2,0,0,2,1,1,...,15,2,0,3,0,1,2,0,0,0.0
d_354,0,1,0,2,0,0,0,7,1,0,...,5,1,0,3,0,0,0,0,0,0.0
d_355,0,0,0,1,2,0,0,0,0,0,...,7,1,0,1,0,1,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_1937,0,0,0,1,0,0,1,5,0,1,...,6,3,0,0,1,0,1,0,0,0.0
d_1938,3,0,2,3,0,0,0,4,0,1,...,4,3,1,2,0,1,0,1,2,0.0
d_1939,3,0,3,0,2,5,1,1,0,0,...,4,3,1,1,0,0,1,1,2,0.0
d_1940,0,0,0,2,1,2,1,40,1,0,...,1,0,0,3,1,1,0,1,5,0.0


In [18]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Assuming dt is a pandas DataFrame
dt.columns = dt.columns.astype(str)  # Convert column names to strings

sc = MinMaxScaler(feature_range=(0, 1))
dt_scaled = sc.fit_transform(dt)

In [19]:
X_train = []
y_train = []
for i in range(timesteps, 1941 - startDay):
    X_train.append(dt_scaled[i-timesteps:i])
    y_train.append(dt_scaled[i][0:30490]) 

In [20]:
del dt_scaled

In [21]:
#Convert to np array to be able to feed the LSTM model
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train.shape)
print(y_train.shape)

(1577, 14, 30491)
(1577, 30490)


In [22]:
X_train_token_ids = np.argmax(X_train, axis=-1)

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Embedding
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
                                     i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
                                     d_model=d_model)
        # Apply sin to even indices in the array; 2i
        sines = tf.math.sin(angle_rads[:, 0::2])
        # Apply cos to odd indices in the array; 2i+1
        cosines = tf.math.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sines, cosines], axis=-1)[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

class TransformerEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerEncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(key_dim=d_model, num_heads=num_heads)
        self.ffn = tf.keras.Sequential([
          Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
          Dense(d_model)  # (batch_size, seq_len, d_model)
        ])

        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)

        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.mha(inputs, inputs, inputs)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2

def build_transformer(num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
    inputs = Input(shape=(None,))
    x = Embedding(input_vocab_size, d_model)(inputs)
    x = PositionalEncoding(maximum_position_encoding, d_model)(x)

    for _ in range(num_layers):
        x = TransformerEncoderLayer(d_model, num_heads, dff, rate)(x)

    x = GlobalAveragePooling1D()(x)
    x = Dropout(rate)(x)
    outputs = Dense(1, activation='linear')(x)  # Assuming a regression task

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Define model parameters
num_layers = 4
d_model = 128
num_heads = 4
dff = 512
input_vocab_size = 30500 
maximum_position_encoding = 1000  # This should cover the maximum sequence length

# Build the Transformer model
transformer = build_transformer(num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding)

# Compile the model
transformer.compile(optimizer=Adam(1e-4), loss='mean_squared_error')

# Summary of the model to verify the architecture
transformer.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 128)         3904000   
                                                                 
 positional_encoding (Posit  (None, None, 128)         0         
 ionalEncoding)                                                  
                                                                 
 transformer_encoder_layer   (None, None, 128)         396032    
 (TransformerEncoderLayer)                                       
                                                                 
 transformer_encoder_layer_  (None, None, 128)         396032    
 1 (TransformerEncoderLayer                                      
 )                                                           

In [24]:
transformer.fit(X_train_token_ids, y_train, epochs=32, batch_size=32)

Epoch 1/32
50/50 [==============================] - 4s 35ms/step - loss: 0.3167
Epoch 2/32
50/50 [==============================] - 2s 39ms/step - loss: 0.0836
Epoch 3/32
50/50 [==============================] - 2s 37ms/step - loss: 0.0561
Epoch 4/32
50/50 [==============================] - 2s 38ms/step - loss: 0.0396
Epoch 5/32
50/50 [==============================] - 2s 38ms/step - loss: 0.0346
Epoch 6/32
50/50 [==============================] - 2s 41ms/step - loss: 0.0321
Epoch 7/32
50/50 [==============================] - 2s 39ms/step - loss: 0.0285
Epoch 8/32
50/50 [==============================] - 2s 40ms/step - loss: 0.0270
Epoch 9/32
50/50 [==============================] - 2s 40ms/step - loss: 0.0256
Epoch 10/32
50/50 [==============================] - 2s 40ms/step - loss: 0.0247
Epoch 11/32
50/50 [==============================] - 2s 39ms/step - loss: 0.0243
Epoch 12/32
50/50 [==============================] - 2s 41ms/step - loss: 0.0233
Epoch 13/32
50/50 [==================

In [25]:
inputs= dt[-timesteps:]
inputs = sc.transform(inputs)

In [27]:
timesteps = 14
input_vocab_size = 30500

# Simulating 'X_test' as a sequence of token IDs
import numpy as np
X_test = np.random.randint(0, input_vocab_size, size=(1, timesteps))

In [29]:
predictions = []

for j in range(28): 
    predicted_stock_price = transformer.predict(X_test[:, -timesteps:])
    new_predicted_token_id = np.random.randint(0, input_vocab_size, size=(1, 1))
    X_test = np.append(X_test, new_predicted_token_id, axis=1)
    predictions.append(new_predicted_token_id)

1/1 [==============================] - 0s 8ms/step


In [30]:
import time

predictions = [np.random.rand(30490) for _ in range(28)]

submission = pd.DataFrame(predictions).T  
submission = pd.concat([submission]*2, ignore_index=True)

sample_submission = pd.read_csv("m5-forecasting-accuracy/sample_submission.csv")
idColumn = sample_submission[["id"]]

idColumn = pd.concat([idColumn]*2, ignore_index=True)

submission.insert(0, "id", idColumn)

submission.columns = ["id"] + [f"F{i}" for i in range(1, 29)]

currentDateTime = time.strftime("%d%m%Y_%H%M%S")

submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.929730,0.316954,0.150438,0.031826,0.728231,0.317111,0.421921,0.466631,0.495570,...,0.677112,0.123201,0.455577,0.318850,0.055633,0.024127,0.284948,0.139579,0.051700,0.366320
1,HOBBIES_1_002_CA_1_validation,0.841432,0.566750,0.684554,0.397764,0.956959,0.181047,0.672897,0.345936,0.691241,...,0.754755,0.023362,0.289381,0.289748,0.168259,0.282514,0.677379,0.832062,0.371777,0.315821
2,HOBBIES_1_003_CA_1_validation,0.417080,0.208739,0.440104,0.656203,0.444759,0.540940,0.912606,0.892679,0.379630,...,0.322693,0.418539,0.503076,0.450961,0.579365,0.615034,0.512169,0.880871,0.483272,0.463457
3,HOBBIES_1_004_CA_1_validation,0.987642,0.106454,0.147299,0.957881,0.522731,0.563162,0.130966,0.226684,0.388966,...,0.938598,0.187533,0.309318,0.888231,0.649323,0.312185,0.364592,0.112490,0.491640,0.518116
4,HOBBIES_1_005_CA_1_validation,0.725440,0.238509,0.211980,0.461397,0.173967,0.177254,0.456350,0.393172,0.785808,...,0.083585,0.550505,0.032918,0.766628,0.499115,0.703054,0.147191,0.067567,0.984402,0.313634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.415392,0.808486,0.505348,0.504575,0.934458,0.863032,0.846521,0.648356,0.500442,...,0.646920,0.333011,0.280607,0.381309,0.072393,0.559686,0.704153,0.252586,0.170317,0.791754
60976,FOODS_3_824_WI_3_evaluation,0.949115,0.054787,0.721572,0.783513,0.810808,0.098465,0.480249,0.847973,0.596691,...,0.828207,0.942006,0.531418,0.411897,0.886838,0.949277,0.397679,0.561155,0.766417,0.208007
60977,FOODS_3_825_WI_3_evaluation,0.173710,0.179358,0.795624,0.502252,0.266719,0.252243,0.564935,0.760476,0.317522,...,0.772316,0.170844,0.485068,0.527790,0.426350,0.119456,0.797439,0.638621,0.069102,0.541170
60978,FOODS_3_826_WI_3_evaluation,0.764811,0.345971,0.141325,0.745247,0.299907,0.659147,0.479449,0.702293,0.313834,...,0.663206,0.963557,0.635876,0.511978,0.506082,0.761189,0.471200,0.538240,0.187174,0.398269


In [31]:
submission.to_csv("submission_transformer.csv", index=False)